In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download e exploração inicial dos dados

In [ ]:
!git clone https://github.com/pubmedqa/pubmedqa.git

import json

file_path = 'pubmedqa/data/ori_pqal.json'

with open(file_path, 'r') as f:
    data = json.load(f)

sample_key = list(data.keys())[0]
print(f"\nCampos disponíveis: {list(data[sample_key].keys())}\n")

print("=" * 60)
print("Exploração de dados - PubMedQA")
print("=" * 60)

for i, key in enumerate(list(data.keys())[:3]):
    item = data[key]

    print(f"\nExemplo {i+1} | ID: {key}")
    print("-" * 60)
    print(f"Question: {item.get('QUESTION', 'N/A')}")

    context = " ".join(item.get('CONTEXTS', []))
    print(f"Context: {context[:300]}...")

    print(f"Labels: {item.get('LABELS', 'N/A')}")
    print(f"Decision: {item.get('final_decision', 'N/A')}")
    print(f"Answer: {item.get('LONG_ANSWER', 'N/A')[:200]}...")
    print(f"Meshes: {item.get('MESHES', 'N/A')}")
    print(f"Year: {item.get('YEAR', 'N/A')}")
    print(f"Reasoning required pred: {item.get('reasoning_required_pred', 'N/A')}")
    print(f"Reasoning free pred: {item.get('reasoning_free_pred', 'N/A')}")

print(f"\n\nTotal de registros: {len(data)}")

# Pré-processamento

In [ ]:
import json
import re
import unicodedata
import os

def preprocess_text(text):
    """Normaliza e limpa texto"""
    if not isinstance(text, str):
        return ""
    
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def map_decision(decision):
    decision = decision.lower()
    if decision == "yes":
        return "SIM"
    elif decision == "no":
        return "NÃO"
    elif decision == "maybe":
        return "TALVEZ"
    return

def preprocess_dataset(input_path, output_path):
    """Pré-processa o dataset original completo"""
    print(f"Carregando {input_path}...")
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = {}

    for key, item in data.items():

        # QUESTION
        question = preprocess_text(item.get('QUESTION', ''))

        # CONTEXTS (string ou lista)
        context_raw = " ".join(item.get('CONTEXTS', []))
        context = preprocess_text(context_raw)

        # DECISION
        decision_raw = item.get('final_decision', 'N/A')
        decision = map_decision(decision_raw)

        # LONG_ANSWER
        long_answer = preprocess_text(item.get('LONG_ANSWER', ''))

        processed_data[key] = {
            "QUESTION": question,
            "CONTEXTS": context,
            "DECISION": decision,
            "LONG_ANSWER": long_answer,
            "YEAR": item.get('YEAR', 'N/A'),
            "ORIGINAL_ID": key
        }

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f_out:
        json.dump(processed_data, f_out, indent=4, ensure_ascii=False)

    print(f"Processados {len(processed_data)} registros")
    return len(processed_data)

# Processar dataset original completo
total = preprocess_dataset(
    'pubmedqa/data/ori_pqal.json',
    'data_processed/ori_pqal_preprocessed.json'
)

print(f"\nPré-processamento concluído: {total} registros")
print("Arquivo salvo em: data_processed/ori_pqal_preprocessed.json")

# Anonimização

In [ ]:
import json
import re
import os

def anonymize_text(text):
    """Remove dados sensíveis (LGPD/HIPAA compliance)"""
    if not isinstance(text, str):
        return ""

    text = re.sub(r'(Dr\.|Dra\.|Doctor|Prof\.|MD)\s+[A-Z][a-z]+(\s+[A-Z][a-z]+)?', '[NOME]', text)
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '[EMAIL]', text)
    locations = r'(Israel|Denmark|Chile|Texas|France|United Kingdom|UK|USA|Pakistan|Karachi|Jordan|Japan|Australia|North Carolina|Washington)'
    text = re.sub(locations, '[LOCAL]', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d{6,}\b', '[ID]', text)
    text = re.sub(r'\b(19|20)\d{2}\b', '[ANO]', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)

    return text

def anonymize_dataset(input_path, output_path):
    """Anonimiza o dataset pré-processado completo mantendo IDs e metadados"""
    print(f"Carregando {input_path}...")
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    anonymized = {}

    for key, item in data.items():
        new_id = f"HOSP_REG_{key[:8]}"  #TODO: Revisar esse esquema de ID anonimizado

        anonymized[new_id] = {
            "QUESTION": anonymize_text(item.get('QUESTION', '')),
            "CONTEXTS": anonymize_text(item.get('CONTEXTS', '')),
            "DECISION": item.get('DECISION', ''),
            "LONG_ANSWER": anonymize_text(item.get('LONG_ANSWER', '')),
            "YEAR": item.get('YEAR', 'N/A'),
            "ORIGINAL_ID": key,
            "SOURCE_DB": "PubMedQA"
        }

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f_out:
        json.dump(anonymized, f_out, indent=4, ensure_ascii=False)

    print(f"Anonimizados {len(anonymized)} registros")
    return len(anonymized)

# Anonimizar dataset pré-processado
total = anonymize_dataset(
    'data_processed/ori_pqal_preprocessed.json',
    'data_anonymized/ori_pqal_anonymized.json'
)

print(f"\nAnonimização concluída: {total} registros")
print("Arquivo salvo em: data_anonymized/ori_pqal_anonymized.json")

# Exemplo de dado anonimizado
with open('data_anonymized/ori_pqal_anonymized.json', 'r', encoding='utf-8') as f:
    sample = json.load(f)
    first_key = list(sample.keys())[0]
    print(f"\n{'='*60}")
    print("Exemplo de dado anonimizado:")
    print(f"{'='*60}")
    print(f"ID: {first_key}")
    print(f"Question: {sample[first_key]['QUESTION'][:100]}...")
    print(f"Source: {sample[first_key]['SOURCE_DB']} | ID: {sample[first_key]['ORIGINAL_ID']} | Ano: {sample[first_key]['YEAR']}")

## Análise de Qualidade

In [ ]:
import json
import os
from collections import Counter

def analyze_quality(data):
    issues = {
        'question_vazia': [],
        'context_vazio': [],
        'decision_vazia': [],
        'context_muito_curto': []
    }

    for key, item in data.items():
        if not item.get('QUESTION', '').strip():
            issues['question_vazia'].append(key)

        if not item.get('CONTEXTS', '').strip():
            issues['context_vazio'].append(key)

        if not item.get('DECISION', '').strip():
            issues['decision_vazia'].append(key)

        if len(item.get('CONTEXTS', '')) < 100:
            issues['context_muito_curto'].append(key)

    return issues

def analyze_distribution(data):
    distribution = Counter()

    for key, item in data.items():
        decision = item.get('DECISION', 'UNKNOWN')
        distribution[decision] += 1

    return distribution

print("Analisando qualidade dos dados...")

with open('/content/data_anonymized/ori_pqal_anonymized.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

issues = analyze_quality(test_data)

print("\nProblemas encontrados:")
for issue_type, ids in issues.items():
    if ids:
        print(f"  {issue_type}: {len(ids)} registros")
    else:
        print(f"  {issue_type}: 0")

print("\nDistribuição:")
dist_test = analyze_distribution(test_data)
for cls, count in dist_test.items():
    pct = (count / len(test_data)) * 100
    print(f"  {cls}: {count} ({pct:.1f}%)")

print(f"\n{'='*60}")
print(f"Total de registros: {len(test_data)}")
print(f"{'='*60}")

## Fine Tunning

Instando as dependências

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

Configuração das variáveis do modelo

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
import os
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


Conversão do dataset para treinamento


In [ ]:
import json
from datasets import Dataset
import os

DATA_PATH = "data_anonymized/ori_pqal_anonymized.json"
OUTPUT_DATA_PATH = "data_final/final_pqal.json"

SYSTEM_PROMPT = """
Você é um assistente médico-científico especializado. Sua função é responder perguntas médicas de forma clara e conversacional, como em um chat.

REGRAS OBRIGATÓRIAS:
- Responda de forma natural e conversacional, como se estivesse explicando para um colega
- Base sua resposta EXCLUSIVAMENTE no contexto científico fornecido
- Não utilize conhecimento externo
- Seja direto e objetivo
- Use linguagem clara e profissional
- A resposta deve começar com SIM, NÃO ou TALVEZ, seguido de uma explicação natural

CRITÉRIOS DE DECISÃO:
- SIM: quando o contexto apoiar claramente a afirmação
- NÃO: quando o contexto claramente contradizer a afirmação
- TALVEZ: quando as evidências forem insuficientes, inconclusivas ou conflitantes

PROIBIÇÕES:
- Não inclua rótulos técnicos, marcadores ou formatação especial
- Não repita a pergunta
- Não faça recomendações de tratamentos ou medicamentos
- Não use listas ou tópicos
- Não mencione "contexto", "análise" ou termos técnicos de processamento

Responda de forma fluida e natural, mantendo o tom profissional mas acessível.
"""

def validate_answer(decision):
    """Valida se a decisão está no formato correto"""
    if decision in ["SIM", "NÃO", "TALVEZ"]:
        return True
    return False

def format_response(decision: str, long_answer: str) -> str:
    """Formata resposta do assistente baseado na decisão - ELIMINA CÓDIGO DUPLICADO"""
    prefixes = {
        "SIM": "Sim.",
        "NÃO": "Não.",
        "TALVEZ": "Talvez."
    }
    
    defaults = {
        "SIM": "Baseado nas evidências disponíveis, a afirmação é correta.",
        "NÃO": "Baseado nas evidências disponíveis, a afirmação não é correta.",
        "TALVEZ": "As evidências disponíveis são inconclusivas ou insuficientes para uma resposta definitiva."
    }
    
    prefix = prefixes.get(decision, "")
    answer = long_answer if long_answer else defaults.get(decision, "")
    
    return f"{prefix} {answer}"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

data = []
for record_id, item in raw_data.items():
    decision = item.get("DECISION", "")
    
    if not validate_answer(decision):
        continue

    response = format_response(decision, item.get('LONG_ANSWER', ''))

    # Adicionar informações de fonte no final da resposta
    source_info = f"\n\n[Fonte: {item.get('SOURCE_DB', 'N/A')} | ID: {item.get('ORIGINAL_ID', 'N/A')} | Ano: {item.get('YEAR', 'N/A')}]"
    response_with_source = response + source_info

    data.append({
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": item["QUESTION"]},
            {"role": "assistant", "content": response_with_source},
        ],
        "metadata": {
            "record_id": record_id,
            "original_id": item.get('ORIGINAL_ID', ''),
            "year": item.get('YEAR', ''),
            "source_db": item.get('SOURCE_DB', '')
        }
    })

formatted_data = Dataset.from_list(data)

print("Novo formato do dataset:")
print(json.dumps(formatted_data[0], indent=2, ensure_ascii=False))

os.makedirs(os.path.dirname(OUTPUT_DATA_PATH), exist_ok=True)
with open(OUTPUT_DATA_PATH, 'w', encoding='utf-8') as output_file:
    json.dump(formatted_data.to_list(), output_file, indent=4, ensure_ascii=False)

print("\n")
print("="*60)
print(f"Dataset final salvo em: {OUTPUT_DATA_PATH}")
print(f"Total de exemplos: {len(data)}")
print("="*60)

## Carregando o modelo "unsloth/llama-3-8b-bnb-4bit"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,  # AUMENTADO de 16 para 32 - maior capacidade de adaptação
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Mantém igual a r (recomendado)
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
from datasets import load_dataset

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    texts = []
    for messages in examples["messages"]:
        full_text = ""
        for msg in messages:
            full_text += msg["content"].strip() + "\n"
        texts.append(full_text.strip() + EOS_TOKEN)
    return {"text": texts}

OUTPUT_PATH_DATASET = "data_final/final_pqal.json"

dataset = load_dataset(
    "json",
    data_files=OUTPUT_PATH_DATASET,
    split="train"
)

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names
)

# SPLIT TRAIN/VALIDATION - 85% treino, 15% validação
dataset_split = dataset.train_test_split(test_size=0.15, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

print(f"Total de exemplos: {len(dataset)}")
print(f"Treino: {len(train_dataset)} exemplos ({len(train_dataset)/len(dataset)*100:.1f}%)")
print(f"Validação: {len(eval_dataset)} exemplos ({len(eval_dataset)/len(dataset)*100:.1f}%)")
print("\nPrimeiro exemplo do dataset de treino:")
print(train_dataset[0]["text"][:500] + "...")

In [ ]:
os.environ["WANDB_DISABLED"] = "false"
os.environ["WANDB_PROJECT"] = "assistente-medico-lora"  #TODO: validar porque por nome do projeto no WANDB
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  #TODO: validar porque salvar checkpoints no WANDB

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,  # Usando dataset de treino
    eval_dataset = eval_dataset,    # Usando dataset de validação
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,  # TODO: Desejável aumentar para 500-1000 em produção
        
        # CONFIGURAÇÕES DE AVALIAÇÃO
        eval_strategy = "steps",  # Avaliar a cada N steps
        eval_steps = 20,          # Avaliar a cada 20 steps
        
        # CHECKPOINTING - salvar progresso
        save_strategy = "steps",
        save_steps = 20,          # Salvar a cada 20 steps
        save_total_limit = 3,     # Manter apenas os 3 melhores checkpoints
        load_best_model_at_end = True,  # Carregar melhor modelo ao final
        metric_for_best_model = "eval_loss",  # Métrica para escolher melhor modelo
        
        # LOGGING
        logging_steps = 1,
        logging_first_step = True,
        report_to = "wandb",  # Enviar métricas para WANDB
        
        # Otimização
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

In [ ]:
from google.colab import drive
from dotenv import load_dotenv
from huggingface_hub import login
import os

# Carregar env
ENV_PATH = "/content/drive/MyDrive/token-hf/env"
load_dotenv(ENV_PATH)

HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

HF_REPO = f"{os.getenv("HF_USER_REPO")}/assistente-medico-lora"
model.push_to_hub(HF_REPO)
tokenizer.push_to_hub(HF_REPO)

# TESTE BÁSICO DO MODELO TREINADO

In [ ]:
test_examples = [
    {
        "question": "O uso diário de protetor solar previne o câncer de pele em pessoas com alto risco de desenvolver a doença?",
        "context": "Estudos longitudinais mostraram que o uso regular e consistente de protetor solar com FPS 30 ou superior, em indivíduos com histórico familiar de melanoma ou com múltiplos nevos atípicos, reduziu significativamente a incidência de novos casos de câncer de pele não melanoma e melanoma em comparação com grupos de controle que usavam protetor solar ocasionalmente ou não usavam. A aplicação correta e reaplicação conforme as instruções são cruciais para a eficácia.",
        "metadata": {
            "source_db": "PubMedQA_Test",
            "original_id": "TEST_001",
            "year": "2024"
        }
    },
    {
        "question": "A dieta cetogênica é recomendada como tratamento primário para hipertensão arterial em todos os pacientes?",
        "context": "A dieta cetogênica tem mostrado resultados promissores na redução da pressão arterial em alguns estudos, especialmente em pacientes com obesidade e resistência à insulina. No entanto, sua segurança e eficácia a longo prazo como tratamento primário para hipertensão em toda a população de pacientes não foram estabelecidas por completo. Diretrizes atuais recomendam uma abordagem individualizada, considerando comorbidades e potencial para efeitos adversos. Alguns estudos indicam que, em pacientes específicos, pode ser uma opção viável sob supervisão médica, enquanto outros alertam para a necessidade de mais pesquisas antes de uma recomendação generalizada.",
        "metadata": {
            "source_db": "PubMedQA_Test",
            "original_id": "TEST_002",
            "year": "2024"
        }
    },
    {
        "question": "A vacina contra a gripe é eficaz em 100% dos casos para prevenir a infecção pelo vírus influenza?",
        "context": "A eficácia da vacina contra a gripe varia anualmente e depende de diversos fatores, como a correspondência entre as cepas da vacina e as que estão em circulação, e a idade e o estado de saúde do indivíduo vacinado. Em geral, a vacina é eficaz em 40% a 60% na prevenção da infecção pelo vírus influenza. Embora não seja 100% eficaz, ela reduz significativamente o risco de desenvolver a doença e suas complicações graves, incluindo hospitalizações e mortes.",
        "metadata": {
            "source_db": "PubMedQA_Test",
            "original_id": "TEST_003",
            "year": "2024"
        }
    },
    {
        "question": "O consumo moderado de café está associado a um risco aumentado de doenças cardiovasculares em adultos saudáveis?",
        "context": "Múltiplos estudos observacionais e meta-análises sugerem que o consumo moderado de café (cerca de 3-4 xícaras por dia) não está associado a um risco aumentado de doenças cardiovasculares em adultos saudáveis. Na verdade, alguns estudos indicam uma possível associação com um risco ligeiramente reduzido de certas condições cardíacas, embora mais pesquisas sejam necessárias para estabelecer causalidade. O consumo excessivo, no entanto, pode estar ligado a efeitos adversos em indivíduos sensíveis.",
        "metadata": {
            "source_db": "PubMedQA_Test",
            "original_id": "TEST_004",
            "year": "2024"
        }
    },
    {
        "question": "O tratamento com antibióticos é sempre necessário para resfriados comuns?",
        "context": "Resfriados comuns são infecções virais das vias aéreas superiores. Antibióticos são medicamentos projetados para combater infecções bacterianas e não têm efeito contra vírus. O uso desnecessário de antibióticos pode levar à resistência a antibióticos, um problema de saúde pública crescente. Portanto, o tratamento com antibióticos não é indicado para resfriados comuns.",
        "metadata": {
            "source_db": "PubMedQA_Test",
            "original_id": "TEST_005",
            "year": "2024"
        }
    }
]

print(f"Gerados {len(test_examples)} exemplos de teste.")

In [ ]:
FastLanguageModel.for_inference(model)

eos_id = tokenizer.eos_token_id

print("\n--- Executando testes com exemplos gerados ---")

for i, example in enumerate(test_examples):
    question = example["question"]
    context = example["context"]
    metadata = example["metadata"]

    prompt = f"""{SYSTEM_PROMPT}

    Pergunta:
    {question}

    Contexto científico:
    {context}

    Resposta:
    """.strip()

    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        temperature=0.0,
        eos_token_id=eos_id,
        repetition_penalty=1.1,
        use_cache=True
    )

    generated_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    response = tokenizer.decode(
        generated_ids,
        skip_special_tokens=True
    ).strip()

    # Adicionar fonte ao final da resposta
    source_info = f"\n\n[Fonte: {metadata['source_db']} | ID: {metadata['original_id']} | Ano: {metadata['year']}]"
    final_response = response + source_info

    print(f"\n{'─'*80}")
    print(f"PERGUNTA {i+1}:")
    print(f"{'─'*80}")
    print(f"{question}\n")
    
    print(f"{'─'*80}")
    print(f"RESPOSTA DO ASSISTENTE:")
    print(f"{'─'*80}")
    print(final_response)
    print("\n" + "="*80)

print("\nTestes concluídos.")
print("="*80)

# Teste com dados reais do dataset

Teste usando exemplos reais do dataset anonimizado

In [ ]:
import json
import random

# Carregar dados reais do dataset anonimizado
DATA_FILE = '/content/data_anonymized/ori_pqal_anonymized.json'

with open(DATA_FILE, 'r', encoding='utf-8') as f:
    real_data = json.load(f)

random_keys = random.sample(list(real_data.keys()), min(5, len(real_data)))

print("="*100)
print("TESTE COM DADOS REAIS DO DATASET")
print("="*100)
print(f"\nTotal de registros disponíveis: {len(real_data)}")
print(f"Testando {len(random_keys)} exemplos aleatórios\n")

FastLanguageModel.for_inference(model)
eos_id = tokenizer.eos_token_id

for idx, key in enumerate(random_keys, 1):
    item = real_data[key]
    
    question = item["QUESTION"]
    context = item["CONTEXTS"]
    expected_decision = item["DECISION"]
    metadata = {
        "source_db": item.get("SOURCE_DB", "N/A"),
        "original_id": item.get("ORIGINAL_ID", "N/A"),
        "year": item.get("YEAR", "N/A")
    }

    prompt = f"""{SYSTEM_PROMPT}

Contexto científico:
{context}

Pergunta:
{question}

Resposta:
""".strip()

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=max_seq_length
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        temperature=0.0,
        eos_token_id=eos_id,
        repetition_penalty=1.1,
        use_cache=True
    )

    generated_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    model_response = tokenizer.decode(
        generated_ids,
        skip_special_tokens=True
    ).strip()

    # Adicionar fonte
    source_info = f"\n\n[Fonte: {metadata['source_db']} | ID: {metadata['original_id']} | Ano: {metadata['year']}]"
    final_response = model_response + source_info

    # Exibir resultado
    print("\n" + "─"*100)
    print(f"TESTE {idx} | ID: {key}")
    print("─"*100)
    print(f"\nPERGUNTA:")
    print(f"{question}")
    
    print(f"\nCONTEXTO (primeiros 300 caracteres):")
    print(f"{context[:300]}...")
    
    print(f"\nDECISÃO ESPERADA (Ground Truth):")
    print(f"{expected_decision}")
    
    print(f"\nRESPOSTA DO MODELO:")
    print(final_response)
    
    # Verificar se a decisão está correta
    model_decision = "DESCONHECIDO"
    if model_response.upper().startswith("SIM"):
        model_decision = "SIM"
    elif model_response.upper().startswith("NÃO") or model_response.upper().startswith("NAO"):
        model_decision = "NÃO"
    elif model_response.upper().startswith("TALVEZ"):
        model_decision = "TALVEZ"
    
    match = "CORRETO" if model_decision == expected_decision else "INCORRETO"
    print(f"\nAVALIAÇÃO: {match}")
    print(f"   Esperado: {expected_decision} | Modelo: {model_decision}")
    print("="*100)

print("\nTeste completo finalizado.")
print("="*100)